In [8]:
###############Query Builder for the FDA Api
import requests
##Base query: https://api.fda.gov/drug/event.json

##query_url = "https://api.fda.gov/drug/event.json?search=receivedate:[20040101+TO+20231207]&count=patient.drug.openfda.pharm_class_epc.exact"

query_url = "https://api.fda.gov/drug/event.json?search=receivedate:[20220101+TO+20231207]&count=patient.drug.medicinalproduct.exact"

res = requests.get(query_url).json()

####Get a full list of all the drug names first. We can use them to look for adverse effects later

names_list = []

for item in range(0,len(res["results"])):
    names_list.append(res["results"][item]["term"])

print(names_list)

####Lovely!



['HUMIRA', 'ENBREL', 'ZANTAC', 'REVLIMID', 'METFORMIN', 'ASPIRIN', 'PREDNISONE.', 'AMLODIPINE', 'METHOTREXATE', 'ATORVASTATIN', 'ASPIRIN.', 'LYRICA', 'DUPIXENT', 'OMEPRAZOLE.', 'PROACTIV MD ADAPALENE ACNE TREATMENT', 'TYSABRI', 'PREDNISONE', 'LIPITOR', 'OXYCONTIN', 'XARELTO', 'NEXIUM', 'SYNTHROID', 'ELIQUIS', 'GABAPENTIN.', 'PANTOPRAZOLE', 'REMICADE', 'METHOTREXATE.', 'LISINOPRIL.', 'VITAMIN D', 'DEXAMETHASONE.', 'PROACTIV MD DEEP CLEANSING FACE WASH', 'AVONEX', 'PROLIA', 'PROACTIV MD DAILY OIL CONTROL SPF 30', 'COSENTYX', 'LASIX', 'OTEZLA', 'FUROSEMIDE.', 'VITAMIN D3', 'CALCIUM', 'RANITIDINE.', 'TYLENOL', 'MIRENA', 'SPIRIVA', 'OMEPRAZOLE', 'TECFIDERA', 'LANTUS', 'CYMBALTA', 'REPATHA', 'CRESTOR', 'LISINOPRIL', 'FOLIC ACID.', 'ACETAMINOPHEN', 'PLAVIX', 'FORTEO', 'SIMVASTATIN.', 'ENTRESTO', 'PRILOSEC', 'FOLIC ACID', 'HUMALOG', 'AVANDIA', 'IBUPROFEN.', 'DEXAMETHASONE', 'VITAMIN B12', 'CELEBREX', 'LEVOTHYROXINE.', 'XELJANZ', 'ORENCIA', 'PARACETAMOL', 'METOPROLOL.', 'FUROSEMIDE', 'XANAX', '

In [9]:
########Now we get the concrete calls:
import pandas as pd 

second_query = 'https://api.fda.gov/drug/event.json?search=(receivedate:[20040101+TO+20231207])+AND+patient.drug.medicinalproduct:"ASPIRIN"&limit=50'

#second_query = 'https://api.fda.gov/drug/event.json?search=(receivedate:[20040101+TO+20231207])+AND+patient.drug.medicinalproduct:"ASPIRIN"&limit=50'

res_02 = requests.get(second_query).json()

#print(res_02["results"][0])

#Trial DF

dataframe_setup = []

for patient in range(0,len(res_02["results"])):
    #print(patient)
    ##Create a new line
    line = []

    ###Simplified: Serious, weight, sex, date, reactions get appended inndeiately without specification
    reactions = []
    for reaction in range(0,len(res_02["results"][patient]["patient"]['reaction'])):
        reactions.append(res_02["results"][patient]["patient"]['reaction'][reaction]['reactionmeddrapt'])
        #print(res_02["results"][patient]["patient"]['reaction'][reaction]['reactionmeddrapt'])

    try:
        line.append(res_02["results"][patient]["serious"])
    except:
        line.append("N/A")
    try:
        line.append(res_02["results"][patient]["patient"]["patientweight"])
    except: 
        line.append("N/A")
    try:
        line.append(res_02["results"][patient]["patient"]["patientsex"])
    except:
        line.append("N/A")
    try:    
        line.append(res_02["results"][patient]["receivedate"])
    except:
        line.append("N/A")
    try:
        line.append(reactions)
    except:
        line.append("N/A")

    #print(line)

    ###Append Line to dataframe_setup
    dataframe_setup.append(line)


#print(dataframe_setup)
#######Create Dataframe:

patients = pd.DataFrame(dataframe_setup, columns =['Serious', 'Weight', "sex", "date", "reactions"]) 

print(patients.head())

reactions = patients["reactions"].tolist()

reactions_list = [j for sub in reactions for j in sub]

final_react_count = [[x,reactions_list.count(x)] for x in set(reactions_list)]

final_react_dict =  {}

for elem in range(0, len(final_react_count)):
    final_react_dict[final_react_count[elem][0]] = final_react_count[elem][1]


sorted_reactions = sorted(final_react_dict.items(), key=lambda x:x[1], reverse=True)
final_react_dict_sorted = dict(sorted_reactions )


print(final_react_count)

print(final_react_dict_sorted)


  Serious Weight sex      date  \
0       2    N/A   2  20140312   
1       2    N/A   2  20140312   
2       1    N/A   2  20140312   
3       1    N/A   2  20140312   
4       2   82.1   1  20140312   

                                           reactions  
0                            [Drug hypersensitivity]  
1                                        [Back pain]  
2  [Cerebrovascular accident, Blood pressure incr...  
3               [Oedema peripheral, Fluid retention]  
4  [Hypertension, Dehydration, Night sweats, Pani...  
[['Abdominal tenderness', 1], ['Coronary arterial stent insertion', 1], ['Drug ineffective', 2], ['Laboratory test abnormal', 1], ['Haemoglobin decreased', 4], ['Dehydration', 1], ['Influenza', 1], ['Off label use', 1], ['Wrong technique in product usage process', 1], ['Drug dose omission', 1], ['Heart rate increased', 1], ['Increased tendency to bruise', 1], ['Asthenia', 1], ['Chest pain', 1], ['Back pain', 1], ['Mobility decreased', 1], ['Nephrolithiasis', 1]

In [7]:
###Easier: Find cases based on symptoms, then suggest drugs
###Query builder

start = "2004"

finish = "2023"

date_range = f"(receivedate:[{start}0101+TO+{finish}1207])"

med = "ASPIRIN"

speed = "&limit=50'"

product = f'+AND+patient.drug.medicinalproduct:"{med}"'

sex_binary = "1"

sex = f'+AND+patient.patientsex:{sex_binary}'

query_builder = f'https://api.fda.gov/drug/event.json?search={date_range}{product}{sex}{speed}'


print(query_builder)

https://api.fda.gov/drug/event.json?search=(receivedate:[20040101+TO+20231207])+AND+patient.drug.medicinalproduct:"ASPIRIN"+AND+patient.patientsex:1&limit=50'


Here is an environment in which you can try stuff out